In [0]:
'''
Russian Universal Sentence Encoder
'''


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title Setup Environment
#latest Tensorflow that supports sentencepiece is 1.13.1
!pip3 uninstall --quiet --yes tensorflow
!pip3 install --quiet tensorflow-gpu==1.13.1
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet sentencepiece
!pip3 install --quiet tf-sentencepiece
!pip3 install --quiet simpleneighbors
!pip3 install --quiet nltk

     |████████████████████████████████| 345.2MB 60kB/s 
     |████████████████████████████████| 3.2MB 29.9MB/s 
     |████████████████████████████████| 368kB 31.7MB/s 
     |████████████████████████████████| 1.0MB 2.7MB/s 
     |████████████████████████████████| 2.7MB 2.7MB/s 
     |████████████████████████████████| 645kB 3.9MB/s 


In [4]:
#@title Setup common imports and functions
import json
import nltk
import os
import pprint
import random
import simpleneighbors
import urllib
from IPython.display import HTML, display

import tensorflow as tf
import tensorflow_hub as hub
import tf_sentencepiece

nltk.download('punkt')


def download_squad(url):
  return json.load(urllib.request.urlopen(url))

def extract_sentences_from_squad_json(squad):
  all_sentences = []
  for data in squad['data']:
    for paragraph in data['paragraphs']:
      sentences = nltk.tokenize.sent_tokenize(paragraph['context'])
      all_sentences.extend(zip(sentences, [paragraph['context']] * len(sentences)))
  return list(set(all_sentences)) # remove duplicates

def extract_questions_from_squad_json(squad):
  questions = []
  for data in squad['data']:
    for paragraph in data['paragraphs']:
      for qas in paragraph['qas']:
        if qas['answers']:
          questions.append((qas['question'], qas['answers'][0]['text']))
  return list(set(questions))

def output_with_highlight(text, highlight):
  output = "<li> "
  i = text.find(highlight)
  while True:
    if i == -1:
      output += text
      break
    output += text[0:i]
    output += '<b>'+text[i:i+len(highlight)]+'</b>'
    text = text[i+len(highlight):]
    i = text.find(highlight)
  return output + "</li>\n"

def display_nearest_neighbors(query_text, answer_text=None):
  query_embedding = session.run(question_embedding, feed_dict={question: [query_text]})['outputs'][0]
  search_results = index.nearest(query_embedding, n=num_results)

  if answer_text:
    result_md = '''
    <p>Random Question from SQuAD:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    <p>Answer:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % (query_text , answer_text)
  else:
    result_md = '''
    <p>Question:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % query_text

  result_md += '''
    <p>Retrieved sentences :
    <ol>
  '''

  if answer_text:
    for s in search_results:
      result_md += output_with_highlight(s, answer_text)
  else:
    for s in search_results:
      result_md += '<li>' + s + '</li>\n'

  result_md += "</ol>"
  display(HTML(result_md))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
for data in predict_file:
    for paragraph in data:
      sentences = nltk.tokenize.sent_tokenize(paragraph)
      print(sentences)
      
  

['/']
['c']
['o']
['n']
['t']
['e']
['n']
['t']
['/']
['d']
['r']
['i']
['v']
['e']
['/']
['M']
['y']
[]
['D']
['r']
['i']
['v']
['e']
['/']
['R']
['u']
['S']
['Q']
['u']
['A']
['D']
['/']
['d']
['e']
['v']
['-']
['v']
['1']
['.']
['1']
['.']
['j']
['s']
['o']
['n']


In [21]:

folderSQuAD = '/content/drive/My Drive/RuSQuAD'
train_file = '/content/drive/My Drive/RuSQuAD/train-v1.1.json'
predict_file = folderSQuAD + '/dev-v1.1.json'

with open(train_file, 'r', encoding='utf-8-sig') as f:
    squad_json = json.load(f)

#squad_json = json.loads(str(train_file))
sentences = extract_sentences_from_squad_json(squad_json)
questions = extract_questions_from_squad_json(squad_json)
print("%s sentences, %s questions extracted from SQuAD %s" % (len(sentences), len(questions)))

print("\nExample sentence and context:\n")
sentence = random.choice(sentences)
print("sentence:\n")
pprint.pprint(sentence[0])
print("\ncontext:\n")
pprint.pprint(sentence[1])
print()

48917 sentences, 45328 questions extracted from SQuAD https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

Example sentence and context:

sentence:

('Заложенный в 1095 году на высоком холме при слиянии рек Трубеж и Лыбедь, '
 'кремль приобрёл свой современный облик к началу XX века, когда закончилось '
 'последнее строительство на его территории.')

context:

('Рязанский кремль — центральное ядро города, архитектурная доминанта его '
 'исторической части и центральная жемчужина Рязанской области. Заложенный в '
 '1095 году на высоком холме при слиянии рек Трубеж и Лыбедь, кремль приобрёл '
 'свой современный облик к началу XX века, когда закончилось последнее '
 'строительство на его территории. Рядом с ним располагается Соборный парк, '
 'комплекс Спасо-Преображенского монастыря, Приокский лесопарк. От Кремлёвской '
 'пристани ежедневно в период судоходства отправляются экскурсионные речные '
 'трамвайчики.')



In [22]:
#@title Set up Tensorflow graph and session
%%time
import tensorflow as tf
import tensorflow_hub as hub
import tf_sentencepiece

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/1"
# Set up graph.
g = tf.Graph()
with g.as_default():
  module = hub.Module(module_url)  # load tfhub module
  question = tf.placeholder(dtype=tf.string, shape=[None])  # question
  response = tf.placeholder(dtype=tf.string, shape=[None])  # response
  response_context = tf.placeholder(
      dtype=tf.string, shape=[None])  # response context
  question_embedding = module(
      question, signature="question_encoder", as_dict=True)
  response_embedding = module(
      inputs={
          "input": response,
          "context": response_context
      },
      signature="response_encoder",
      as_dict=True)
  init_op = tf.group(
      [tf.global_variables_initializer(),
       tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 36.9 s, sys: 3.17 s, total: 40 s
Wall time: 41.9 s


In [23]:
#@title Compute embeddings and build simpleneighbors index
%%time
from google.colab import output

batch_size = 100
encodings = session.run(
    response_embedding,
    feed_dict={
        response: [sentences[0][0]],
        response_context: [sentences[0][1]]
    })
index = simpleneighbors.SimpleNeighbors(
    len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(sentences))
slices = zip(*(iter(sentences),) * batch_size)
num_batches = int(len(sentences) / batch_size)
for n, s in enumerate(slices):
  output.clear(output_tags='progress')
  with output.use_tags('progress'):
    print('Processing batch %s of %s' % (n + 1, num_batches))
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = session.run(
      response_embedding,
      feed_dict={
          response: response_batch,
          response_context: context_batch
      })
  for i in range(len(response_batch)):
    index.add_one(response_batch[i], encodings['outputs'][i])

index.build()
print('simpleneighbors index for %s sentences built.' % len(sentences))

Computing embeddings for 48917 sentences


Processing batch 489 of 489


simpleneighbors index for 48917 sentences built.
CPU times: user 1h 40min 25s, sys: 1min 55s, total: 1h 42min 21s
Wall time: 52min 48s


In [47]:
#@title Retrieve nearest neighbors for a random question from SQuAD
num_results = 5 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(questions)
#query = questions[0]
display_nearest_neighbors(query[0], query[1])

In [0]:
index.save('/content/drive/My Drive/RuSQuAD')

In [55]:
indexm1 = simpleneighbors.SimpleNeighbors(512)
indexm1 = indexm1.load('/content/drive/My Drive/RuSQuAD')
print(index.dims)

512


In [76]:
def display_nearest_neighborsm1(query_text, answer_text=None):
  query_embedding = session.run(question_embedding, feed_dict={question: [query_text]})['outputs'][0]
  search_results = indexm1.nearest(query_embedding, n=num_results)

  if answer_text:
    result_md = '''
    <p>Random Question from SQuAD:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    <p>Answer:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % (query_text , answer_text)
  else:
    result_md = '''
    <p>Question:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % query_text

  result_md += '''
    <p>Retrieved sentences :
    <ol>
  '''

  if answer_text:
    for s in search_results:
      result_md += output_with_highlight(s, answer_text)
  else:
    for s in search_results:
      result_md += '<li>' + s + '</li>\n'

  result_md += "</ol>"
  display(HTML(result_md))
  
  
num_results = 5 

query = random.choice(questions)
#query = questions[0]
display_nearest_neighborsm1(query[0], query[1])

query_embedding = session.run(question_embedding, feed_dict={question: [query[0]]})['outputs'][0]
print(query_embedding.shape)
search_results = index.nearest(query_embedding, n=num_results)
print(search_results)
search_resultsm1 = indexm1.nearest(query_embedding, n=num_results)
print(search_resultsm1)

(512,)
['Хотя это и не является убеждением главенствующей экономической теории, ряд центральных банкиров и экономистов кредитно-денежной политики верят в то, что банковский мультипликатор или модель перекредитования, не является реалистичным способом описания того, как банки работают в реальности и нежели чем депозиты приводят к кредитам, обратная причинная связь, кредиты приводят к депозитам.', 'Хоум Кредит начал переходить от модели банка-монолайнера к универсальному розничному банку.', 'Известны следующие факты о банке в то время: получение в декабре 1993 года расширенной валютной лицензии, позволяющей (помимо осуществления операций с валютой на территории России (внутренняя валютная лицензия)) установить корреспондентские отношения с шестью иностранными банками; объявление об обнаружении в июле 1994 года, наряду с некоторыми другими кредитными организациями, фальшивых банковских документов (гарантийных писем); судебный процесс в 1996 году с банком Автосельхозмаш из-за задолженности